In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio as img

import prepare

from common import *
from testing import accuracy
from testing import mse
from testing import mae

In [2]:
def generator():
  random_value = np.random.normal(mean, std, (1,))
  random_value = np.round(random_value).astype(np.uint16)
  random_value = random_value[0]
  return random_value

In [3]:
train, test = data_sets()
count_test = test[2]
count_test = count_test.astype(np.uint16)

In [4]:
coo, counts = prepare.read_anwsers("mall_dataset/mall_gt.mat")

mean = np.mean(counts)
std = np.std(counts)

In [5]:
iterations = 100000

results = np.empty((iterations, 3), dtype=np.float32)

for i in range(iterations):
  predictions = np.array([generator() for x in range(count_test.shape[0])])
  
  count_test = count_test.astype(np.int16)
  predictions = predictions.astype(np.int16)
  
  diff = predictions - count_test
  
  results[i, 0] = np.mean(accuracy(predictions, count_test)) * 100
  results[i, 1] = mse(diff)
  results[i, 2] = mae(diff)

In [6]:
mean_results = np.mean(results, axis=0)
std_results = np.std(results, axis=0)

In [7]:
trustfulness = 0.95
trustfulness = 1 - trustfulness
alpha = 1 - trustfulness / 2 * std_results / np.sqrt(iterations)

In [8]:
print(alpha)

[0.9999196  0.99946517 0.99997616]


In [9]:
# Quantiles from https://planetcalc.com/4987/
quantiles = np.array([3.773768, 3.271533, 4.066717])

In [10]:
confidence_interval = np.stack([mean_results, mean_results])
confidence_interval[0, :] -= quantiles
confidence_interval[1, :] += quantiles

In [13]:
print(f"Przedział ufności dla trafności wynosi: {confidence_interval[:, 0]}")
print(f"Przedział ufności dla MSE wynosi: {confidence_interval[:, 1]}")
print(f"Przedział ufności dla MAE wynosi: {confidence_interval[:, 2]}")

Przedział ufności dla trafności wynosi: [69.99109 77.53862]
Przedział ufności dla MSE wynosi: [92.22858 98.77164]
Przedział ufności dla MAE wynosi: [ 3.7414591 11.874893 ]
